In [ ]:
import pandas as pd
import numpy as np
import os
import pandas as pd
import numpy as np
import os

data_dir = r'D:\Learning (H)\研究生\WUSTL\Semester 3\Practicum in Data Analytics & Statistics ESE 527\Project\Data' 



#获取所有患者住院信息，从adm文件提取所有患者记录病去重，只保留了第一次
def get_all_patients():  
    admissions = pd.read_csv(os.path.join(data_dir, 'ADMISSIONS.csv'),
                             usecols=['SUBJECT_ID', 'HADM_ID'],
                             dtype={'SUBJECT_ID': int, 'HADM_ID': int})
    
    admissions = admissions.drop_duplicates(subset=['SUBJECT_ID'], keep='first')
    return admissions

#获取所有患者demographic特征和住院特征

def get_demographics(hadm_ids):
    patients = pd.read_csv(os.path.join(data_dir, 'PATIENTS.csv'),
                           parse_dates=['DOB'],
                           dtype={'SUBJECT_ID': int})

    admissions = pd.read_csv(os.path.join(data_dir, 'ADMISSIONS.csv'),
                             parse_dates=['ADMITTIME', 'DISCHTIME', 'DEATHTIME'],
                             dtype={'SUBJECT_ID': int, 'HADM_ID': int, 'HOSPITAL_EXPIRE_FLAG': int})

    icustays = pd.read_csv(os.path.join(data_dir, 'ICUSTAYS.csv'),
                           parse_dates=['INTIME', 'OUTTIME'],
                           dtype={'SUBJECT_ID': int, 'HADM_ID': int, 'ICUSTAY_ID': int})

    icustays = icustays.sort_values(by=['SUBJECT_ID', 'INTIME']).drop_duplicates(subset=['SUBJECT_ID'], keep='first')

    icustays['LOS_ICU'] = (icustays['OUTTIME'] - icustays['INTIME']).dt.total_seconds() / 86400
    icustays = icustays[icustays['LOS_ICU'] >= 1]  #过滤掉住院天数小于一天的

    admissions = admissions[admissions['HADM_ID'].isin(hadm_ids)]
    icustays = icustays[icustays['HADM_ID'].isin(hadm_ids)]

    demographics = admissions.merge(patients, on='SUBJECT_ID', how='left')
    demographics = demographics.merge(icustays, on=['SUBJECT_ID', 'HADM_ID'], how='left')
    
    demographics['DOB'] = pd.to_datetime(demographics['DOB'], errors='coerce').dt.year   
    ADMITTIME_var= pd.to_datetime(demographics['ADMITTIME'], errors='coerce').dt.year
    demographics['AGE'] = (ADMITTIME_var - demographics['DOB'])  #计算患者年龄

    demographics.loc[demographics['AGE'] < 0, 'AGE'] = np.nan  #小于0岁的定义为nan
    demographics.loc[demographics['AGE'] > 300, 'AGE'] = 90  #300的定义为90岁
    
    demographics['GENDER'] = demographics['GENDER'].map({'M': 1, 'F': 0})  

    ethnicity_map = {
        'ASIAN': 'Asian',
        'WHITE': 'White',
        'BLACK': 'Black',
        'HISPANIC': 'Hispanic',
        'UNKNOWN/NOT SPECIFIED':'Unknown'
    }
    demographics['ETHNICITY'] = demographics['ETHNICITY'].str.upper()
    demographics['ETHNICITY'] = demographics['ETHNICITY'].apply(
        lambda x: next((v for k, v in ethnicity_map.items() if k in x), 'Other')
    )
    demographics['ETHNICITY'] = demographics['ETHNICITY'].fillna('Unknown')

    demographics['LOS_HOSPITAL'] = (demographics['DISCHTIME'] - demographics['ADMITTIME']).dt.total_seconds() / 86400
    demographics['LOS_ICU'] = (demographics['OUTTIME'] - demographics['INTIME']).dt.total_seconds() / 86400 #ICU住了多久

    demographics = demographics[[
        'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'GENDER', 'ETHNICITY', 'AGE',
        'LOS_HOSPITAL', 'LOS_ICU', 'HOSPITAL_EXPIRE_FLAG'
    ]].drop_duplicates()

    return demographics


#提取特定的vital signs
def get_vital_signs(icustay_ids):
    vital_signs_items = {
        'HeartRate': [211, 220045],
        'SysBP': [51, 442, 455, 6701, 220179, 220050],
        'DiasBP': [8368, 8441, 8555, 220180, 220051],
        'MeanBP': [52, 6702, 443, 456, 220181, 225312],
        'RespRate': [618, 615, 220210, 224690],
        'Temperature': [223761, 678, 676, 223762],
        'SpO2': [646, 220277],
        'CVP': [113, 220074],
        'MAP': [456, 220181],
        'GCS': [198, 223900]
    }

    item_ids = []
    item_labels = []
    for label, ids in vital_signs_items.items():  #用于分别生成ids和labels表，表里的数字和label能一一对应
        item_ids.extend(ids)
        item_labels.extend([label] * len(ids))

    itemid_label_map = pd.DataFrame({'ITEMID': item_ids, 'VITAL_SIGN': item_labels})
    vital_signs_list = []

    chunksize = 10 ** 6
    for chunk in pd.read_csv(os.path.join(data_dir, 'CHARTEVENTS.csv'),
                             chunksize=chunksize,
                             parse_dates=['CHARTTIME'],  #时间列
                             usecols=['ICUSTAY_ID', 'ITEMID', 'CHARTTIME', 'VALUENUM'], #指定读取的列
                             dtype={'ICUSTAY_ID': float, 'ITEMID': int, 'VALUENUM': float}):  #指定数据类型
        chunk = chunk[chunk['ICUSTAY_ID'].isin(icustay_ids)]
        chunk = chunk[chunk['ITEMID'].isin(item_ids)]
        chunk = chunk.merge(itemid_label_map, on='ITEMID', how='left')
        chunk = chunk.dropna(subset=['VALUENUM'])
        vital_signs_list.append(chunk)

    if vital_signs_list:
        vital_signs = pd.concat(vital_signs_list, ignore_index=True)
    else:
        vital_signs = pd.DataFrame(columns=['ICUSTAY_ID', 'ITEMID', 'CHARTTIME', 'VALUENUM', 'VITAL_SIGN'])

    icustays = pd.read_csv(os.path.join(data_dir, 'ICUSTAYS.csv'),
                           parse_dates=['INTIME', 'OUTTIME'],
                           dtype={'ICUSTAY_ID': int})

    icustays = icustays[icustays['ICUSTAY_ID'].isin(icustay_ids)][['ICUSTAY_ID', 'INTIME', 'OUTTIME']]

    vital_signs = vital_signs.merge(icustays, on='ICUSTAY_ID', how='left')

    # Calculate time difference to select data from 72 to 24 hours before discharge/death
    vital_signs['HOURS_TO_END'] = (vital_signs['OUTTIME'] - vital_signs['CHARTTIME']).dt.total_seconds() / 3600
    vital_signs = vital_signs[(vital_signs['HOURS_TO_END'] >= 24) & (vital_signs['HOURS_TO_END'] <= 72)]

    agg_vital_signs = vital_signs.groupby(['ICUSTAY_ID', 'VITAL_SIGN'])['VALUENUM'].agg(['mean', 'min', 'max']).reset_index()
    pivot_vital_signs = agg_vital_signs.pivot(index='ICUSTAY_ID', columns='VITAL_SIGN')
    pivot_vital_signs.columns = ['_'.join(col).strip() for col in pivot_vital_signs.columns.values]
    pivot_vital_signs = pivot_vital_signs.reset_index()

    return pivot_vital_signs

#  Extract laboratory values from the first 24 hours of ICU stay
def get_lab_values(hadm_ids):
    lab_items = {
        'WBC': [51300],
        'Hemoglobin': [51222],
        'Hematocrit': [51221],
        'Platelets': [51265],
        'Glucose': [50931],
        'Creatinine': [50912],
        'BUN': [51006],
        'Sodium': [50983],
        'Potassium': [50971],
        'Chloride': [50902],
        'CO2': [50882],
        'Bilirubin': [50885],
        'Albumin': [50862],
        'AnionGap': [50868],
        'Lactate': [50813],
        'Calcium': [50893],
        'Magnesium': [50960],
        'Phosphate': [50970]
    }

    item_ids = []
    item_labels = []
    for label, ids in lab_items.items():
        item_ids.extend(ids)
        item_labels.extend([label] * len(ids))

    itemid_label_map = pd.DataFrame({'ITEMID': item_ids, 'LAB_TEST': item_labels})

    lab_values_list = []

    chunksize = 10 ** 6
    for chunk in pd.read_csv(os.path.join(data_dir, 'LABEVENTS.csv'),
                             chunksize=chunksize,
                             parse_dates=['CHARTTIME'],
                             usecols=['HADM_ID', 'ITEMID', 'CHARTTIME', 'VALUENUM'],
                             dtype={'HADM_ID': float, 'ITEMID': int, 'VALUENUM': float}):
        chunk = chunk[chunk['HADM_ID'].isin(hadm_ids)]
        chunk = chunk[chunk['ITEMID'].isin(item_ids)]
        chunk = chunk.merge(itemid_label_map, on='ITEMID', how='left')
        chunk = chunk.dropna(subset=['VALUENUM'])

        lab_values_list.append(chunk)

    # Concatenate all chunks
    if lab_values_list:
        lab_values = pd.concat(lab_values_list, ignore_index=True)
    else:
        lab_values = pd.DataFrame(columns=['HADM_ID', 'ITEMID', 'CHARTTIME', 'VALUENUM', 'LAB_TEST'])

    admissions = pd.read_csv(os.path.join(data_dir, 'ADMISSIONS.csv'),
                             parse_dates=['ADMITTIME'],
                             dtype={'HADM_ID': int})

    admissions = admissions[admissions['HADM_ID'].isin(hadm_ids)][['HADM_ID', 'ADMITTIME']]

    lab_values = lab_values.merge(admissions, on='HADM_ID', how='left')

    lab_values['HOURS_FROM_ADMIT'] = (lab_values['CHARTTIME'] - lab_values['ADMITTIME']).dt.total_seconds() / 3600

    # lab_values = lab_values[(lab_values['HOURS_FROM_ADMIT'] >= 0) & (lab_values['HOURS_FROM_ADMIT'] <= 24)]

    agg_lab_values = lab_values.groupby(['HADM_ID', 'LAB_TEST'])['VALUENUM'].agg(['mean', 'min', 'max']).reset_index()
    pivot_lab_values = agg_lab_values.pivot(index='HADM_ID', columns='LAB_TEST')
    pivot_lab_values.columns = ['_'.join(col).strip() for col in pivot_lab_values.columns.values]
    pivot_lab_values = pivot_lab_values.reset_index()

    return pivot_lab_values

# Extract interventions (mechanical ventilation, renal replacement therapy)
def get_interventions(icustay_ids, hadm_ids):
    # Mechanical Ventilation
    mv_itemids = [720, 721, 722, 223848, 223849]

    mv_list = []

    chunksize = 10 ** 6
    for chunk in pd.read_csv(os.path.join(data_dir, 'CHARTEVENTS.csv'),
                             chunksize=chunksize,
                             usecols=['ICUSTAY_ID', 'ITEMID'],
                             dtype={'ICUSTAY_ID': float, 'ITEMID': int}):
        chunk = chunk[chunk['ICUSTAY_ID'].isin(icustay_ids)]
        chunk = chunk[chunk['ITEMID'].isin(mv_itemids)]
        mv_list.append(chunk[['ICUSTAY_ID']].drop_duplicates())

    if mv_list:
        mv_df = pd.concat(mv_list, ignore_index=True).drop_duplicates()
        mv_df['MECHANICAL_VENTILATION'] = 1
    else:
        mv_df = pd.DataFrame(columns=['ICUSTAY_ID', 'MECHANICAL_VENTILATION'])

    # Renal Replacement Therapy (RRT)
    procedures_icd = pd.read_csv(os.path.join(data_dir, 'PROCEDURES_ICD.csv'),
                                 dtype={'ICD9_CODE': str, 'HADM_ID': int})

    rrt_icd9_codes = ['39.95', '54.98', '38.95']
    rrt_df = procedures_icd[procedures_icd['ICD9_CODE'].isin(rrt_icd9_codes)]
    rrt_df = rrt_df[['HADM_ID']].drop_duplicates()
    rrt_df['RENAL_REPLACEMENT_THERAPY'] = 1

    # Extracorporeal Membrane Oxygenation (ECMO)
    ecmo_icd9_code = ['39.65']
    ecmo_df = procedures_icd[procedures_icd['ICD9_CODE'].isin(ecmo_icd9_code)]
    ecmo_df = ecmo_df[['HADM_ID']].drop_duplicates()
    ecmo_df['ECMO'] = 1

    # Invasive Monitoring (Central Venous Catheter, Arterial Line)
    invasive_monitoring_icd9_codes = ['38.93', '38.91']
    invasive_monitoring_df = procedures_icd[procedures_icd['ICD9_CODE'].isin(invasive_monitoring_icd9_codes)]
    invasive_monitoring_df = invasive_monitoring_df[['HADM_ID']].drop_duplicates()
    invasive_monitoring_df['INVASIVE_MONITORING'] = 1

    # Load ICUSTAYS.csv to map ICUSTAY_IDs to HADM_IDs
    icustays = pd.read_csv(os.path.join(data_dir, 'ICUSTAYS.csv'),
                           usecols=['ICUSTAY_ID', 'HADM_ID'],
                           dtype={'ICUSTAY_ID': int, 'HADM_ID': int})
    interventions = pd.DataFrame({'ICUSTAY_ID': icustay_ids})
    interventions = interventions.merge(icustays, on='ICUSTAY_ID', how='left')

    # Merge all interventions with default fill values of 0
    interventions = interventions.merge(mv_df, on='ICUSTAY_ID', how='left').fillna({'MECHANICAL_VENTILATION': 0})
    interventions = interventions.merge(rrt_df, on='HADM_ID', how='left').fillna({'RENAL_REPLACEMENT_THERAPY': 0})
    interventions = interventions.merge(ecmo_df, on='HADM_ID', how='left').fillna({'ECMO': 0})
    interventions = interventions.merge(invasive_monitoring_df, on='HADM_ID', how='left').fillna({'INVASIVE_MONITORING': 0})

    # Select only the required columns
    interventions = interventions[['ICUSTAY_ID', 'MECHANICAL_VENTILATION', 'RENAL_REPLACEMENT_THERAPY', 'ECMO', 'INVASIVE_MONITORING']]

    return interventions

# Step 6: Extract comorbidities (e.g., diabetes, malignant tumor)
def get_comorbidities(hadm_ids):
    # ICD-9 codes for comorbidities
    comorbidity_codes = {
        'DIABETES': ['25000', '25002', '25040', '25060', '25090'],
        'MALIGNANT_TUMOR': ['1400', '20936', '20970', '20974', '20975', '20979'],
        'CHF': ['4280', '4281', '42820', '42821', '42822', '42823'],  # Congestive Heart Failure
        'COPD': ['49121', '4928', '496'],  # Chronic Obstructive Pulmonary Disease
        'HYPERTENSION': ['4011', '4019'],  # Hypertension, unspecified
        'CKD': ['5853', '5854', '5855', '5856', '5859'],  # Chronic Kidney Disease
        'LIVER_DISEASE': ['5712', '5715', '5716', '5718'],  # Liver Disease
        'OBESITY': ['27800', '27801', '2781']  # Obesity
    }

    # Read DIAGNOSES_ICD.csv
    diagnoses_icd = pd.read_csv(os.path.join(data_dir, 'DIAGNOSES_ICD.csv'),
                                dtype={'ICD9_CODE': str, 'HADM_ID': int})

    comorbidities = pd.DataFrame({'HADM_ID': hadm_ids})

    # For each comorbidity, check if any relevant ICD-9 codes are present in the patient's records
    for comorbidity, codes in comorbidity_codes.items():
        comorbidity_df = diagnoses_icd[diagnoses_icd['ICD9_CODE'].isin(codes)]
        comorbidity_df = comorbidity_df[['HADM_ID']].drop_duplicates()
        comorbidity_df[comorbidity] = 1  # Mark presence of the comorbidity
        comorbidities = comorbidities.merge(comorbidity_df, on='HADM_ID', how='left')

    # Fill missing values with 0 to indicate absence of the comorbidity
    comorbidities = comorbidities.fillna(0)

    return comorbidities

# Step 7: Merge all data into a single DataFrame
def merge_data(demographics, vital_signs, lab_values, interventions, comorbidities):
    # Merge on ICUSTAY_ID and HADM_ID as appropriate
    data = demographics.merge(vital_signs, on='ICUSTAY_ID', how='left')
    data = data.merge(interventions, on='ICUSTAY_ID', how='left')
    data = data.merge(lab_values, on='HADM_ID', how='left')
    data = data.merge(comorbidities, on='HADM_ID', how='left')

    # Fill missing values  NEED REVISION HERE!!!
    # data = data.fillna(0)

    return data

def main():
    # Step 1: Get all patients
    all_patients = get_all_patients()
    hadm_ids = all_patients['HADM_ID'].unique()
    subject_ids = all_patients['SUBJECT_ID'].unique()

    # Step 2: Get Demographics
    demographics = get_demographics(hadm_ids)
    icustay_ids = demographics['ICUSTAY_ID'].dropna().astype(int).unique()

    # Step 3: Get Vital Signs from the last 72-24 hours before discharge or death
    vital_signs = get_vital_signs(icustay_ids)

    # Step 4: Get Lab Values (including expanded list of lab tests)
    lab_values = get_lab_values(hadm_ids)

    # Step 5: Get Interventions (Mechanical Ventilation, RRT, ECMO, Invasive Monitoring)
    interventions = get_interventions(icustay_ids, hadm_ids)

    # Step 6: Get Comorbidities (expanded list of comorbidities relevant to ICU mortality)
    comorbidities = get_comorbidities(hadm_ids)

    # Step 7: Merge all data into a single DataFrame
    final_data = merge_data(demographics, vital_signs, lab_values, interventions, comorbidities)

    # Step 8: Save the final dataset to a CSV file
    final_data.to_csv(r'D:\Learning (H)\研究生\WUSTL\Semester 3\Practicum in Data Analytics & Statistics ESE 527\Project\1114\all_patients_data_with_features.csv', index=False)


In [4]:
main()

In [5]:
data=pd.read_csv(r'D:\Learning (H)\研究生\WUSTL\Semester 3\Practicum in Data Analytics & Statistics ESE 527\Project\1114\all_patients_data_with_features.csv')

In [6]:
data.replace([np.inf, -np.inf], np.nan, inplace=True)
missing_percentage = data.isnull().mean(axis=1)
data=data[missing_percentage <= 0.2]

def one_hot_encode_non_binary(df):
    # Identify columns that are categorical and non-binary (more than two unique values)
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    non_binary_cols = [col for col in categorical_cols if df[col].nunique() > 2]

    # Perform one-hot encoding only on non-binary columns
    df_encoded = pd.get_dummies(df, columns=non_binary_cols, drop_first=True)
    
    return df_encoded
data=one_hot_encode_non_binary(data)
numerical_cols = data.select_dtypes(include=['float64', 'int64']).columns
    
    # Impute missing values with the mean for each numerical column

# data[numerical_cols] = data[numerical_cols].apply(lambda col: col.fillna(col.mean()), axis=0)
data.loc[data['AGE'] >= 90, 'AGE'] =90
data=data[data['AGE']>=18]

data['mean_Temperature'] = data['mean_Temperature'].apply(
            lambda x: x * 9 / 5 + 32 if x < 50 else x
        )

data['max_Temperature'] = data['max_Temperature'].apply(
            lambda x: x * 9 / 5 + 32 if x < 50 else x
        )

data['min_Temperature'] = data['min_Temperature'].apply(
            lambda x: x * 9 / 5 + 32 if x < 50 else x
        )

In [8]:

def create_30_day_icu_mortality_flag(icustays, admissions):
    # Merge icustays and admissions to get DEATHTIME
    merged = icustays.merge(admissions[['SUBJECT_ID', 'HADM_ID', 'DEATHTIME']], on=['SUBJECT_ID', 'HADM_ID'], how='left')

    # Only consider the first ICU admission for each patient
    merged = merged.sort_values(by=['SUBJECT_ID', 'INTIME']).drop_duplicates(subset=['SUBJECT_ID'], keep='first')

    # Calculate the time difference between ICU admission (INTIME) and death/discharge
    merged['DAYS_TO_DEATH'] = (merged['DEATHTIME'] - merged['INTIME']).dt.total_seconds() / 86400  # Convert to days

    # Create a 30-day ICU mortality flag (1 if death within 30 days, 0 otherwise)
    merged['EXPIRE_FLAG_30D'] = np.where((merged['DAYS_TO_DEATH'] <= 30) & (merged['DAYS_TO_DEATH'] >= 0), 1, 0)

    return merged[['ICUSTAY_ID', 'EXPIRE_FLAG_30D']]


data_dir = r'D:\Learning (H)\研究生\WUSTL\Semester 3\Practicum in Data Analytics & Statistics ESE 527\Project\Data' 

patients = pd.read_csv(os.path.join(data_dir, 'PATIENTS.csv'),
                        parse_dates=['DOB'],
                        dtype={'SUBJECT_ID': int})

admissions = pd.read_csv(os.path.join(data_dir, 'ADMISSIONS.csv'),
                            parse_dates=['ADMITTIME', 'DISCHTIME', 'DEATHTIME'],
                            dtype={'SUBJECT_ID': int, 'HADM_ID': int})

icustays = pd.read_csv(os.path.join(data_dir, 'ICUSTAYS.csv'),
                        parse_dates=['INTIME', 'OUTTIME'],
                        dtype={'SUBJECT_ID': int, 'HADM_ID': int, 'ICUSTAY_ID': int})

# Filter to only consider the first ICU stay for each patient
icustays = icustays.sort_values(by=['SUBJECT_ID', 'INTIME']).drop_duplicates(subset=['SUBJECT_ID'], keep='first')

# Exclude ICU stays under 24 hours
icustays['LOS_ICU'] = (icustays['OUTTIME'] - icustays['INTIME']).dt.total_seconds() / 86400
icustays = icustays[icustays['LOS_ICU'] >= 1]  # Keep ICU stays >= 24 hours

expire_30D=create_30_day_icu_mortality_flag(icustays,admissions)


In [9]:
data=data.merge(expire_30D, on='ICUSTAY_ID', how='left')

In [10]:
data.to_csv(r'D:\Learning (H)\研究生\WUSTL\Semester 3\Practicum in Data Analytics & Statistics ESE 527\Project\1114\all_patients_data_with_features.csv2', index=False)

In [12]:
def detect_extreme_outliers(df, iqr_multiplier=3):
    """
    Detect extreme outliers in numeric variables whose column names begin with 'max', 'min', or 'mean' using the IQR method.

    Parameters:
    df (pd.DataFrame): DataFrame to analyze.
    iqr_multiplier (float): The multiplier for the IQR to define extreme outliers. Default is 3.

    Returns:
    dict: A dictionary containing details about the outliers in each matching numeric column.
    """
    # Select numeric columns that start with 'max', 'min', or 'mean'
    selected_columns = [col for col in df.select_dtypes(include=['float64', 'int64']).columns if col.startswith(('max', 'min', 'mean'))]
    
    outliers_dict = {}

    for col in selected_columns:
        # Drop missing values for the calculation
        column_data = df[col].dropna()

        if column_data.empty:
            continue  # Skip columns with all NaN values

        # Calculate Q1, Q3, and IQR
        Q1 = column_data.quantile(0.25)
        Q3 = column_data.quantile(0.75)
        IQR = Q3 - Q1

        # Define the extreme outlier bounds
        lower_bound = Q1 - iqr_multiplier * IQR
        upper_bound = Q3 + iqr_multiplier * IQR

        # Detect extreme outliers
        extreme_outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]

        num_outliers = len(extreme_outliers)

        # Only add columns with outliers to the dictionary
        if num_outliers > 0:
            outliers_dict[col] = {
                'lower_bound': lower_bound,
                'upper_bound': upper_bound,
                'num_outliers': num_outliers,
                'outliers': extreme_outliers
            }
    
    return outliers_dict

def impute_extreme_outliers(df, iqr_multiplier=3):
    """
    Impute extreme outliers in numeric features with their nearest valid (non-outlier) value.

    Parameters:
    df (pd.DataFrame): DataFrame to analyze and impute extreme outliers.
    iqr_multiplier (float): The multiplier for the IQR to define extreme outliers. Default is 3.

    Returns:
    pd.DataFrame: DataFrame with extreme outliers imputed with nearest valid values.
    """
    # Detect extreme outliers
    outliers_info = detect_extreme_outliers(df, iqr_multiplier)
    
    df_imputed = df.copy()
    
    for col, info in outliers_info.items():
        lower_bound = info['lower_bound']
        upper_bound = info['upper_bound']
        
        # Replace values below the lower bound with the nearest valid lower value
        df_imputed[col] = df_imputed[col].apply(
            lambda x: lower_bound if x < lower_bound else upper_bound if x > upper_bound else x
        )
        
        # Ensure values are within the bounds by using the nearest valid values
        non_outliers = df_imputed[(df_imputed[col] >= lower_bound) & (df_imputed[col] <= upper_bound)][col]
        
        # Impute outliers with the nearest valid non-outlier values
        df_imputed[col] = df_imputed[col].apply(
            lambda x: non_outliers.min() if x < lower_bound else non_outliers.max() if x > upper_bound else x
        )

    return df_imputed


In [13]:
from sklearn.impute import KNNImputer

df_cleaned = impute_extreme_outliers(data)

imputer = KNNImputer(n_neighbors=5) 
df_cleaned[numerical_cols] = imputer.fit_transform(df_cleaned[numerical_cols])

In [14]:
df_cleaned

df_cleaned.to_csv(r'D:\Learning (H)\研究生\WUSTL\Semester 3\Practicum in Data Analytics & Statistics ESE 527\Project\1114\all_patients_data_with_features.csv', index=False)

In [29]:
df_cleaned["mean_Temperature"].min()

92.76666666666667

In [30]:
df_cleaned

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,GENDER,AGE,LOS_HOSPITAL,LOS_ICU,HOSPITAL_EXPIRE_FLAG,mean_CVP,mean_DiasBP,...,HYPERTENSION,CKD,LIVER_DISEASE,OBESITY,ETHNICITY_Black,ETHNICITY_Hispanic,ETHNICITY_Other,ETHNICITY_Unknown,ETHNICITY_White,EXPIRE_FLAG_30D
0,22,165315,204798.0,0,65.0,1.144444,1.143750,0,12.680018,73.666667,...,1.0,0.0,0.0,0.0,0,0,0,0,1,0
1,23,152223,227807.0,1,71.0,5.496528,1.264074,0,12.680018,58.461538,...,1.0,0.0,0.0,0.0,0,0,0,0,1,0
2,25,129635,203487.0,1,59.0,3.534028,3.546574,0,12.680018,54.283019,...,1.0,0.0,0.0,0.0,0,0,0,0,1,0
3,26,197661,244882.0,1,72.0,6.988889,2.140683,0,12.680018,42.136364,...,0.0,0.0,0.0,0.0,0,0,0,1,0,0
4,28,162569,225559.0,1,74.0,5.364583,1.122407,0,12.680018,45.500000,...,0.0,0.0,0.0,0.0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30372,98768,127022,213468.0,0,85.0,1.940972,2.151123,1,12.680018,69.733333,...,1.0,0.0,0.0,0.0,0,0,0,0,1,1
30373,98769,141860,233981.0,0,81.0,10.794444,7.788854,0,12.680018,67.083333,...,1.0,0.0,0.0,0.0,0,0,0,0,1,0
30374,98794,190603,262848.0,1,78.0,4.145833,2.288796,0,12.680018,57.140000,...,0.0,0.0,0.0,0.0,0,0,0,0,1,0
30375,98797,105447,244147.0,1,88.0,0.662500,1.238171,1,12.680018,82.000000,...,1.0,0.0,0.0,0.0,0,0,0,0,1,1
